In [1]:
import pandas as pd

from hardware.opentrons.http_communications import OpentronsAPI

from hardware.cameras import PendantDropCamera

from hardware.sensor.sensor_api import SensorAPI
from hardware.opentrons.droplet_manager import DropletManager

from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter

from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model


opentrons_api = OpentronsAPI()

opentrons_api.initialise()

config = Configuration(http_api=opentrons_api)

labware = config.load_labware()

containers = config.load_containers()

pipettes = config.load_pipettes()

left_pipette = pipettes["left"]

right_pipette = pipettes["right"]

pendant_drop_camera = PendantDropCamera()
sensor_api = SensorAPI()
plotter = Plotter()
settings = load_settings()
logger = Logger(
    name="protocol",
    file_path=f'experiments/{settings["EXPERIMENT_NAME"]}/meta_data',
)
# droplet_manager = DropletManager(
#     left_pipette=left_pipette,
#     containers=containers,
#     pendant_drop_camera=pendant_drop_camera,
#     opentrons_api=opentrons_api,
#     plotter=plotter,
#     logger=logger

# )
# droplet_manager.set_max_retries(retries=1)

c:\Users\pimda\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-04 15:03:20,984 - INFO - Protocol already uploaded, using existing protocol.
2025-03-04 15:03:28,119 - INFO - Run created succesfully (ID: ad63eff5-0628-416b-bc48-01d92f14f811).
2025-03-04 15:03:29,923 - INFO - All custom labware definitions added.
2025-03-04 15:03:30,804 - INFO - Labware loaded successfully.
2025-03-04 15:03:30,850 - INFO - Containers loaded successfully.
2025-03-04 15:03:34,972 - INFO - Pipettes loaded successfully.


In [3]:
learner = ActiveLearner(
    model=szyszkowski_model, parameters=["cmc", "gamma_max", "Kad"], logger=logger
)
results = pd.read_csv("results.csv")
suggest_c, suggest_st = learner.suggest(results=results, solution_name="SDS_1")

sample: 100%|██████████| 1500/1500 [00:01<00:00, 1322.73it/s, 31 steps of size 3.25e-02. acc. prob=0.85] 
2025-03-04 15:05:15,276 - INFO - analysis: fitting model to data
2025-03-04 15:05:15,482 - INFO - analysis: calculating bayesian suggestion


In [11]:
formulate_exploit_concentration(suggest_c = suggest_c, containers = containers, solution_name="SDS_1", well_volume=200)

l
r


In [ ]:
def formulate_exploit_concentration(suggest_c: float, containers: dict, solution_name: str, well_volume: float):
    id = get_well_id_concentration(
        containers=containers, solution=solution_name, requested_concentration=suggest_c
    )
    ratio = suggest_c / float(containers[id].concentration)
    v_source = ratio * well_volume
    v_water = well_volume-v_source
    list_v = [v_source, v_water]
    for v in list_v:
        if v < 20:
            left_pipette
        elif v < 1000:
            print("r")
        else:
            print("error")

In [14]:
opentrons_api.home()

2025-03-03 16:17:10,092 - INFO - Robot homed.


In [7]:
drop_parameters = {
    "drop_volume": 11,
    "max_measure_time": 60,
    "flow_rate": 0.5,
}
st_t, drop_parameters = droplet_manager.measure_pendant_drop(
    source = containers["7A1"],
    drop_parameters=drop_parameters
)

2025-03-03 13:40:10,397 - INFO - Picked up tip.
2025-03-03 13:40:10,397 - INFO - Start measurment of pendant drop of 7A1 with drop volume 11 uL and drop count 1.
2025-03-03 13:40:18,830 - INFO - Done with mixing in 7A1 with order before, with volume 15 uL, repeated 3 times
2025-03-03 13:40:22,281 - INFO - Container: aspirated 15 uL from this container with content n.a. mM empty.
2025-03-03 13:40:22,281 - INFO - Aspirated 15 uL from 7A1 with left pipette.
2025-03-03 13:40:26,031 - INFO - Air gap of 5 uL performed in left pipette.
2025-03-03 13:40:33,101 - INFO - Pipette tip cleaned on sponge.
2025-03-03 13:40:38,281 - INFO - Air gap of 5 uL removed in left pipette.
2025-03-03 13:41:01,313 - INFO - Camera: start measuring 7A1.
2025-03-03 13:42:01,914 - INFO - Camera: stopped measurement
2025-03-03 13:42:02,850 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-03 13:42:06,482 - INFO - Container: dispensed 15 uL into this container from source 7A1 containing n.a. mM empty.
2025-

In [15]:
left_pipette.pick_up_tip()

2025-03-03 16:19:51,556 - INFO - Picked up tip.


In [16]:
left_pipette.aspirate(
    volume=15,
    source=containers["7A1"],
)

2025-03-03 16:19:56,576 - WARNING - Aspiration volume is larger than container volume!
2025-03-03 16:19:56,576 - INFO - Aspirated 15 uL from 7A1 with left pipette.


In [17]:
left_pipette.dispense(
    volume=12,
    destination=containers["drop_stage"],
    flow_rate=1,
    depth_offset=-23.4
)

2025-03-03 16:20:50,576 - INFO - Dispensed 12 uL into well 10A1 with left pipette.


In [18]:
left_pipette.drop_tip()

2025-03-03 16:21:22,230 - INFO - Left pipette dropped tip into trash.
